In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
haskey(ENV, "LD_LIBRARY_PATH") && @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "DataFrames",
    "FASTX",
    "XAM",
    "uCSV",
    "CodecZlib",
    "ProgressMeter",
    "StatsBase",
    "Statistics",
    "CSV",
    "Random",
    "Distributions",
    "Plots",
    "OrderedCollections",
    "StatsPlots",
    "Colors"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
basedir = dirname(pwd())
data_dir = joinpath(basedir, "data")

In [ ]:
RUN_ID = "r64342e_20240621_140056"

In [ ]:
# copy over
# run(`rclone copy locus_data_warehouse:Genomics/SequelIIe/$(RUN_ID) $(data_dir)/$(RUN_ID)`)

In [ ]:
xml = first(filter(x -> occursin(r"\.run\.metadata\.xml", x), readdir(joinpath(data_dir, RUN_ID, "1_A01"), join=true)))

In [ ]:
sample_to_barcode_table = Mycelia.extract_pacbiosample_information(xml)
sample_to_barcode_table = sample_to_barcode_table[map(x -> occursin(r"urine"i, x), sample_to_barcode_table[!, "BioSampleName"]), :]

In [ ]:
barcode_directories = filter(x -> occursin(r"^bc\d+", basename(x)) && (basename(x) in Set(sample_to_barcode_table[!, "BarcodeName"])), readdir(joinpath(data_dir, RUN_ID, "1_A01"), join=true))

In [ ]:
# map to blast NT
blast_db = "nt"
blast_dbs_dir = joinpath([homedir(), "workspace", "blastdb"])
# path_to_db = joinpath(homedir(), "workspace", "blastdb", blast_db)
blast_db_path = joinpath(blast_dbs_dir, blast_db)

# # path_to_db = Mycelia.download_blast_db(db=db, source="ncbi")
# # compressed_fasta_export = Mycelia.export_blast_db(path_to_db = path_to_db)
# compressed_fasta_export = blast_db_path * ".fna.gz"

In [ ]:
# # map to Locus internal genomes
# # locus_fasta = "/global/homes/c/cjprybol/workspace/Mycelia/projects/metagenome/data/locus-internal.fna.gz"

# locus_remote_drive = "locus_genomics_storefront:Reference-Genomes/_Bacterial-Strains/Escherichia"
# folder_contents = readlines(`rclone lsf $(locus_remote_drive)`)
# c_strain_folders = filter(x -> occursin(r"^c\d{6}", x), folder_contents)
# locus_c_strain_directory = mkpath(joinpath(data_dir, "locus-c-strains"))

# # didn't work - used them all!!
# # ENV["JULIA_NUM_THREADS"] = 4
# # Threads.@threads 
# for c_strain_folder in c_strain_folders
#     dir = joinpath(locus_remote_drive, c_strain_folder)
#     folder_contents = readlines(`rclone lsf $(dir)`)
#     fastas = filter(x -> occursin(Mycelia.FASTA_REGEX, x), folder_contents)
#     # display(folder_contents)
#     for fasta in fastas
#         local_file = joinpath(locus_c_strain_directory, fasta)
#         if !isfile(local_file)
#             display(fasta)
#             remote_file = dir * fasta
#             run(`rclone copy $(remote_file) $(locus_c_strain_directory)`)
#             @assert isfile(local_file)
#         end
#     end
# end
# in_fastas = filter(x -> occursin(Mycelia.FASTA_REGEX, x), readdir(locus_c_strain_directory, join=true))
# locus_c_strain_fasta = joinpath(data_dir, "locus-c-strains.fna")
# Mycelia.merge_fasta_files(fasta_files=in_fastas, fasta_file=locus_c_strain_fasta)

locus_c_strain_fasta = joinpath(data_dir, "locus-c-strains.fna")

In [ ]:
# # here I made a copy of the fastq file to try submitting to the premium queue
# # premium queue jumps straight to the front of the line, but caps @ 5 submissions

# for d in barcode_directories
#     bam_file = first(filter(x -> occursin(r"\.bam", x), readdir(d, join=true)))
#     # 5655.810957 seconds
#     # 111.551031 seconds (70.00 k allocations: 4.719 MiB, 0.11% compilation time)
#     fastq_file = Mycelia.bam_to_fastq(bam=bam_file)

#     cmd, outfile = Mycelia.minimap_map(fasta=locus_c_strain_fasta, fastq=fastq_file, mem_gb=Mycelia.NERSC_MEM, mapping_type="map-hifi", threads=Mycelia.NERSC_CPU, as_string=true)
#     # display(outfile)
#     # display(cmd)
#     # display(outfile)
#     if !isfile(outfile)
#         println(outfile)
#         Mycelia.nersc_sbatch_premium(
#             job_name=basename(outfile),
#             mail_user="cameron.prybol@gmail.com",
#             logdir=mkpath("$(homedir())/workspace/slurmlogs"),
#             mem_gb=Mycelia.NERSC_MEM,
#             cpus_per_task=Mycelia.NERSC_CPU,
#             cmd=cmd)
#     end
# end

In [ ]:
# trigger analysis

In [ ]:
# find bam files and read them in - use standard microbiome taxonomy analysis from 

In [ ]:
barcode_to_xam = Dict()
for barcode_directory in barcode_directories
    barcode = basename(barcode_directory)
    xams = filter(x -> occursin(Mycelia.XAM_REGEX, x) && occursin("locus-c-strains.fna", x), readdir(barcode_directory, join=true))
    xam = first(xams)
    # println("$(barcode)\t$(xam)")
    barcode_to_xam[barcode] = xam
end
barcode_to_xam
sample_to_barcode_table[!, "xam"] = [barcode_to_xam[barcode] for barcode in sample_to_barcode_table[!, "BarcodeName"]]
sample_to_barcode_table

In [ ]:
# println(first(xams))

In [ ]:
xams = sample_to_barcode_table[!, "xam"]

In [ ]:
# @time blast_db_taxonomy_table = Mycelia.load_blast_db_taxonomy_table(Mycelia.export_blast_db_taxonomy_table(path_to_db = blast_db_path))

In [ ]:
# alignment_score

In [ ]:
# taxa_level = "species"
# file_to_taxa_relative_abundances = OrderedCollections.OrderedDict{String, Dict{Union{Missing, String}, Float64}}()
joint_record_table = DataFrames.DataFrame()
ProgressMeter.@showprogress for xam in xams
    @time record_table = Mycelia.parse_xam_to_mapped_records_table(xam)
    record_table = record_table[record_table[!, "isprimary"], :]
    record_table[!, "top_hit_strain"] = map(x -> lowercase(match(r"(c\d{6})"i, x).captures[1]), record_table[!, "reference"])
    record_table[!, "xam"] .= xam
    append!(joint_record_table, record_table)
    # file_to_taxa_relative_abundances[xam] =  Mycelia.normalize_countmap(StatsBase.countmap(record_table[!, "top_hit_strain"]))
end
# file_to_taxa_relative_abundances
joint_record_table

In [ ]:
grouped_df = DataFrames.groupby(joint_record_table, "top_hit_strain")

In [ ]:
filtered_groups = filter(x -> DataFrames.nrow(x) >= 3, grouped_df)

ys = []
groups = []

for (i, g) in enumerate(filtered_groups)
    push!(groups, g[1, "top_hit_strain"])
    values = g[!, "alignment_score"]
    push!(ys, [values])
end

ordering = sortperm(Statistics.median.(ys), rev=true)
ys = ys[ordering]
groups = groups[ordering]

In [ ]:
ys_sampled = []
for y in ys    
    push!(ys_sampled, StatsBase.sample(y, min(length(y), 1_000), replace=false)...)
end
ys_sampled

In [ ]:
xs = []
for (i, y) in enumerate(ys_sampled)
    push!(xs, fill(i, length(y)))
end
xs

In [ ]:
p = StatsPlots.scatter(
    xs,
    ys,
    ylabel = "Alignment score",
    xlabel = "strain",
    title = "Alignment scores by strain",
    legend=false,
    xticks = (1:length(groups), groups),
    xrotation = 90,
    margins = 10StatsPlots.Plots.PlotMeasures.mm,
    leftmargin = 20StatsPlots.Plots.PlotMeasures.mm,
    size = (2500, 500)
)
StatsPlots.savefig(p, "alignment-score-by-strain.png")
StatsPlots.savefig(p, "alignment-score-by-strain.svg")
display(p)